# Five-area DEC RevBayes

https://revbayes.github.io/tutorials/biogeo/biogeo_simple.html

![schematic](Ortho_earth_scenarios.pdf "schematic")

### Set working directory and input and output filenames

In [4]:
#getwd()
setwd("/Users/cdoorenweerd/Documents/StackIP/Teaching and courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography")

range_fn = "saturnia.n5.range.nex"
tree_fn = "20180606_Saturnia_BEAST_constrained2_ingroup_nocopaxa.nwk"
dist_fn = "saturnia.n5.distances.txt"
connectivity_fn = "saturnia.n5.connectivity.1.txt"

out_fn = "five_area_DEC"
out_state_fn = out_fn + ".states.log"
out_tree_fn = out_fn + ".tre"
out_mcc_fn = out_fn + ".mcc.tre"

mvi = 1
mni = 1

Received directory:		/Users/cdoorenweerd/Documents/StackIP/Teaching and courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography


### Read in geographical ranges

In [5]:
dat_range_01 = readDiscreteCharacterData(range_fn) # read character data as binary presence-absence
n_areas = dat_range_01.nchar() # record total number of areas
dat_range_01

max_areas <- 2
n_states <- 0
for (k in 0:max_areas) n_states += choose(n_areas, k)

dat_range_n = formatDiscreteCharacterData(dat_range_01, "DEC", n_states) # encode ranges into natural numbers for one character
n_areas
n_states

   Successfully read one character matrix from file 'saturnia.n5.range.nex'

   Standard character matrix with 33 taxa and 5 characters
   Origination:                   saturnia.n5.range.nex
   Number of taxa:                33
   Number of included taxa:       33
   Number of characters:          5
   Number of included characters: 5
   Datatype:                      Standard

   5
   16


Write legend for state labels and range states to file

In [6]:
state_desc = dat_range_n.getStateDescriptions()

state_desc_str = "state,range\n"
for (i in 1:state_desc.size()){state_desc_str += (i-1) + "," + state_desc[i] + "\n"}
write(state_desc_str, file=out_fn+".state_labels.txt")

### Read in dated molecular phylogenetic tree

In [7]:
tree <- readTrees(tree_fn)[1]
tree

   Attempting to read the contents of file "20180606_Saturnia_BEAST_constrained2_ingroup_nocopaxa.nwk"
   Successfully read file
  
   (((((((((((SARBA615_09_Rinaca_chinensis[&index=33]:3.513580,SARBC471_10_Rinaca_ruda_HQ973326[&index=32]:3.513580)[&index=34]:1.239046,SAUPA465_10_Rinaca_naumanni_JN278744[&index=31]:4.752626)[&index=35]:4.110972,(SARBA578_09_Rinaca_minshanensis[&index=30]:4.057900,SARBC468_10_Rinaca_florianii_HQ973323[&index=29]:4.057900)[&index=36]:4.805698)[&index=37]:0.625938,(((SARBA597_09_Rinaca_fujiana[&index=28]:1.349058,SARBA613_09_Rinaca_shaanxiana[&index=27]:1.349058)[&index=38]:3.796523,SARBE1506_13_Rinaca_dusii[&index=26]:5.145581)[&index=39]:1.342367,act16_Rinaca_jonasii[&index=25]:6.487948)[&index=40]:3.001588)[&index=41]:1.296271,act18_Rinaca_boisduvalii[&index=24]:10.785807)[&index=42]:0.391241,SARBB798_09_Rinaca_thibeta_GU664031[&index=23]:11.177048)[&index=43]:2.703537,(SASNB457_09_Rinaca_lesoudieri_GU702972[&index=22]:6.207512,SAWNA102_09_Rinaca_zulei

### Build DEC rate matrices and transition probability matrix

In [5]:
# read connectivity matrix
connectivity <- readDataDelimitedFile(file=connectivity_fn, delimiter=" ")

# read distances matrix
distances <- readDataDelimitedFile(file=dist_fn, delimiter=" ")

# parameter for arrival rate of anagenetic range evolution events
log10_rate_bg ~ dnUniform(-4,2)
log10_rate_bg.setValue(-2)
rate_bg := 10^log10_rate_bg
moves[mvi++] = mvSlide(log10_rate_bg, weight=4)

# dispersal rate matrix
dispersal_rate <- 1.0 # fix base rate of dispersal
distance_scale ~ dnUnif(0,20)
distance_scale.setValue(0.01)
moves[mvi++] = mvScale(distance_scale, weight=3)

for (j in 1:n_areas) {
  for (k in 1:n_areas) {
    dr[j][k] <- 0.0
    if (connectivity[j][k] > 0) {
      dr[j][k]  := dispersal_rate * exp(-distance_scale * distances[j][k])
    }
  }
}

# prior distribution for the relative extirpation rate and assign a move
log_sd <- 0.5
log_mean <- ln(1) - 0.5*log_sd^2
extirpation_rate ~ dnLognormal(mean=log_mean, sd=log_sd)
moves[2] = mvScale(extirpation_rate, weight=2)

for (j in 1:n_areas) {
  for (k in 1:n_areas) {
    er[j][k] <- 0.0
  }
  er[j][j] := extirpation_rate
}

# unify objects
Q_DEC := fnDECRateMatrix(dispersalRates=dr, extirpationRates=er, maxRangeSize=max_areas) # relative matrix

# cladogenetic transition probability matrix
clado_event_types <- [ "s", "a" ]
clado_event_probs <- simplex(1, 1)
P_DEC := fnDECCladoProbs(eventProbs=clado_event_probs,
                            eventTypes=clado_event_types,
                            numCharacters=n_areas,
                            maxRangeSize=max_areas)

# encapsulate all model components
m_bg ~ dnPhyloCTMCClado(tree=tree,
                           Q=Q_DEC,
                           cladoProbs=P_DEC,
                           branchRates=rate_bg,
                           nSites=1,
                           type="NaturalNumbers")

# attach observed ranges to the model
m_bg.clamp(dat_range_n)

### Run model

In [6]:
# add monitors that keep track of the MCMC
monitors[1] = mnScreen(rate_bg, extirpation_rate, printgen=100)
monitors[2] = mnModel(file=out_fn+".params.log", printgen=10)
monitors[3] = mnFile(tree, file=out_fn+".tre", printgen=10)
monitors[4] = mnJointConditionalAncestralState(tree=tree,
                                                    ctmc=m_bg,
                                                    filename=out_fn+".states.log",
                                                    type="NaturalNumbers",
                                                    printgen=10,
                                                    withTips=true,
                                                    withStartStates=true)

In [7]:
mymodel = model(m_bg)

In [8]:
mymcmc = mcmc(mymodel, moves, monitors)

In [9]:
mymcmc.run(25000)


   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 2 different moves in a random move schedule with 6 moves per iteration

Iter        |      Posterior   |     Likelihood   |          Prior   |   extirpatio..   |        rate_bg   |    elapsed   |        ETA   |
------------------------------------------------------------------------------------------------------------------------------------------
0           |       -31.2563   |       -26.4929   |        -4.7634   |      0.6925181   |           0.01   |   00:00:00   |   --:--:--   |
100         |       -31.2892   |       -25.1905   |       -6.09863   |        1.55599   |    0.003445192   |   00:00:15   |   --:--:--   |
200         |       -31.4848   |       -25.6799   |        -5.8049   |       1.414351   |    0.002278871   |   00:00:30   |   01:02:00   |
300         |       -32.0842   |       -25.0953   |       -6.98896   |       1.973705   |    0.003175423   |   00:00:44   |   01:00:22

5200        |       -31.7871   |       -26.8619   |       -4.92524   |      0.9135334   |    0.001160476   |   00:12:37   |   00:48:02   |
5300        |       -31.6446   |       -26.0005   |       -5.64402   |       1.334559   |    0.001842399   |   00:12:51   |   00:47:45   |
5400        |       -34.2415   |       -24.4662   |       -9.77535   |       3.346939   |    0.004261901   |   00:13:06   |   00:47:32   |
5500        |       -30.6362   |       -25.7326   |       -4.90367   |      0.5273258   |     0.00475431   |   00:13:20   |   00:47:16   |
5600        |       -30.5912   |       -25.8236   |       -4.76755   |      0.6562525   |    0.002837654   |   00:13:34   |   00:46:59   |
5700        |       -30.8065   |       -25.8533   |       -4.95322   |      0.5050134   |    0.003157226   |   00:13:49   |   00:46:46   |
5800        |       -31.4713   |       -26.0213   |       -5.44991   |      0.3825377   |    0.002820113   |   00:14:04   |   00:46:33   |
5900        |       -30.486

10500       |       -31.6247   |        -26.177   |       -5.44763   |       1.233625   |    0.001675305   |   00:25:49   |   00:35:39   |
10600       |       -30.4593   |       -25.6953   |         -4.764   |      0.6743625   |     0.00604416   |   00:26:04   |   00:35:24   |
10700       |       -34.6691   |       -29.8904   |        -4.7787   |      0.6295282   |     0.01996679   |   00:26:19   |   00:35:10   |
10800       |       -31.8845   |       -26.5493   |       -5.33524   |       1.173238   |    0.001337167   |   00:26:34   |   00:34:55   |
10900       |       -32.5684   |       -26.4547   |       -6.11369   |      0.3021879   |    0.001870571   |   00:26:49   |   00:34:41   |
11000       |       -31.4588   |       -25.9826   |       -5.47622   |       1.248635   |    0.001908136   |   00:27:05   |   00:34:28   |
11100       |       -30.8849   |       -25.1997   |       -5.68511   |       1.355142   |    0.003942749   |   00:27:20   |   00:34:13   |
11200       |       -30.734



Iter        |      Posterior   |     Likelihood   |          Prior   |   extirpatio..   |        rate_bg   |    elapsed   |        ETA   |
------------------------------------------------------------------------------------------------------------------------------------------
16000       |        -30.377   |       -25.5777   |       -4.79931   |      0.7860101   |    0.005892555   |   00:39:34   |   00:22:15   |
16100       |       -32.1956   |       -25.3385   |        -6.8571   |       1.912066   |     0.00898479   |   00:39:50   |   00:22:01   |
16200       |       -30.5292   |       -25.3198   |       -5.20945   |       1.102208   |    0.005665544   |   00:40:04   |   00:21:45   |
16300       |       -31.3797   |       -25.9468   |       -5.43297   |      0.3853294   |    0.003171258   |   00:40:19   |   00:21:31   |
16400       |       -31.0868   |       -25.2759   |       -5.81086   |       1.417271   |    0.007080679   |   00:40:34   |   00:21:16   |
16500       |       -30.5

21300       |       -30.5234   |       -25.3506   |       -5.17289   |       1.080638   |    0.005871991   |   00:52:32   |   00:09:07   |
21400       |       -30.9986   |        -26.177   |       -4.82156   |      0.8152167   |    0.009290595   |   00:52:46   |   00:08:52   |
21500       |       -30.9856   |       -25.6211   |       -5.36459   |       1.189243   |    0.002595769   |   00:53:00   |   00:08:37   |
21600       |       -30.9174   |       -26.0166   |       -4.90081   |      0.8933557   |    0.008935207   |   00:53:14   |   00:08:22   |
21700       |       -33.1639   |       -25.0305   |       -8.13344   |       2.517096   |     0.00285923   |   00:53:28   |   00:08:07   |
21800       |       -31.0158   |       -25.0913   |       -5.92448   |       1.472515   |    0.004509054   |   00:53:44   |   00:07:53   |
21900       |       -31.2641   |       -25.6276   |       -5.63654   |       1.330797   |    0.008743738   |   00:53:58   |   00:07:38   |


Iter        |      Poster

### Write results to output files

In [10]:
tree_trace = readTreeTrace(file=out_tree_fn, treetype="clock")
tree_trace.setBurnin(0.25) # gratuitous when phylogeny is fixed
n_burn = tree_trace.getBurnin()

mcc_tree = mccTree(tree_trace, file=out_mcc_fn)
state_trace = readAncestralStateTrace(file=out_state_fn)
tree_trace = readAncestralStateTreeTrace(file=out_tree_fn, treetype="clock")

anc_tree = ancestralStateTree(tree=mcc_tree,
                              ancestral_state_trace_vector=state_trace,
                              tree_trace=tree_trace,
                              include_start_states=true,
                              file=out_tree_fn+".ase.tre",
                              burnin=n_burn,
                              site=1)

   Processing file "/Users/cdoorenweerd/Documents/StackIP/Projects active/Saturnia/Saturnia_Biogeography/RevBayes_DEC_5area/five_area_DEC.tre"

Progress:
0---------------25---------------50---------------75--------------100
********************************************************************

   Compiling maximum clade credibility tree from 2501 trees in tree trace, using a burnin of 625 trees.
   Summarizing clades ...

Progress:
0---------------25---------------50---------------75--------------100
********************************************************************

   Annotating tree ...
Processing file "/Users/cdoorenweerd/Documents/StackIP/Projects active/Saturnia/Saturnia_Biogeography/RevBayes_DEC_5area/five_area_DEC.states.log"
Processing file "/Users/cdoorenweerd/Documents/StackIP/Projects active/Saturnia/Saturnia_Biogeography/RevBayes_DEC_5area/five_area_DEC.tre"
   Compiling MAP ancestral states from 2501 samples in the ancestral state trace, using a burnin of 625 samples.
  

### Calculate marginal likelihood

Using stepping-stone simulation and path sampling

In [11]:
pow_p = powerPosterior(mymodel, moves, monitors, "model.out", cats=50) 
pow_p.burnin(generations=10000,tuningInterval=1000)


Running burn-in phase of Power Posterior sampler for 10000 iterations.
The simulator uses 2 different moves in a random move schedule with 6 moves per iteration


Progress:
0---------------25---------------50---------------75--------------100
********************************************************************


In [12]:
pow_p.run(generations=1000)


Running power posterior analysis ...
Step  1 / 51		****************************************
Step  2 / 51		****************************************
Step  3 / 51		****************************************
Step  4 / 51		****************************************
Step  5 / 51		****************************************
Step  6 / 51		****************************************
Step  7 / 51		****************************************
Step  8 / 51		****************************************
Step  9 / 51		****************************************
Step 10 / 51		****************************************
Step 11 / 51		****************************************
Step 12 / 51		****************************************
Step 13 / 51		****************************************
Step 14 / 51		****************************************
Step 15 / 51		****************************************
Step 16 / 51		****************************************
Step 17 / 51		****************************************
Step 18 / 51		*************

In [13]:
ss = steppingStoneSampler(file="model.out", powerColumnName="power", likelihoodColumnName="likelihood")

In [14]:
ss.marginal()

   -27.06624


In [15]:
ps = pathSampler(file="model.out", powerColumnName="power", likelihoodColumnName="likelihood")

In [16]:
ps.marginal() 

   -27.06222
